In [52]:
from PIL import Image
import requests
from transformers import AutoProcessor, AutoModel
import torch
import os 
from transformers.models.siglip import SiglipVisionModel
cert = r"Zscaler Root CA.crt"
os.environ["REQUESTS_CA_BUNDLE"] = cert
vision_model = SiglipVisionModel.from_pretrained("google/siglip-base-patch16-256-multilingual")
vision_processor = AutoProcessor.from_pretrained("google/siglip-base-patch16-256-multilingual")

loading configuration file config.json from cache at C:\Users\F4L5GPL\.cache\huggingface\hub\models--google--siglip-base-patch16-256-multilingual\snapshots\8952a4eafcde3cb7ab46b1dd629b33f8784ca9c6\config.json
Model config SiglipVisionConfig {
  "attention_dropout": 0.0,
  "hidden_act": "gelu_pytorch_tanh",
  "hidden_size": 768,
  "image_size": 256,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-06,
  "model_type": "siglip_vision_model",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "transformers_version": "4.44.2"
}

loading weights file model.safetensors from cache at C:\Users\F4L5GPL\.cache\huggingface\hub\models--google--siglip-base-patch16-256-multilingual\snapshots\8952a4eafcde3cb7ab46b1dd629b33f8784ca9c6\model.safetensors
Some weights of the model checkpoint at google/siglip-base-patch16-256-multilingual were not used when initializing SiglipVisionModel: ['logit_bias', 'logit_scale', 'text_model.embeddings.position_embeddi

In [53]:
from PIL import Image

# url = "giay-ra-vien.jpg"
image = Image.open("hoa_don.png")
print(image)
# texts = ["a photo of 2 cats", "a photo of 2 dogs"]
inputs = vision_processor(images=image, padding="max_length", return_tensors="pt")

with torch.no_grad():
    outputs = vision_model(**inputs)
print("Shape: ", outputs.last_hidden_state.size())


<PIL.PngImagePlugin.PngImageFile image mode=RGB size=1199x649 at 0x235EEE736D0>
Shape:  torch.Size([1, 256, 768])


In [70]:
vision_emb = outputs.last_hidden_state
print(vision_emb.size())

torch.Size([1, 256, 768])


In [89]:
from transformers import AutoModel, BartForConditionalGeneration
import torch.nn as nn

class BartPhoExtractor(nn.Module):
    def __init__(self):
        super(BartPhoExtractor, self).__init__()
        self.bartpho_word = BartForConditionalGeneration.from_pretrained("vinai/bartpho-word")
    
    def get_embedding(self, input_ids, attention_mask):
        return self.bartpho_word.get_encoder()(input_ids, attention_mask)
    
    def forward(self, input_ids, attention_mask):
        last_hidden_states = self.bartpho_word(input_ids, attention_mask)
        features = last_hidden_states[0]
        return features
    
language_model = BartPhoExtractor()

loading configuration file config.json from cache at C:\Users\F4L5GPL\.cache\huggingface\hub\models--vinai--bartpho-word\snapshots\bee56ed0f2723cf4510ad3775cf4c5a38eb5e6fa\config.json
You are using a model of type mbart to instantiate a model of type bart. This is not supported for all configurations of models and can yield errors.
Model config BartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "MBartModel"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
 

In [90]:
prompt = "Bệnh nhân tên là gì?"

In [91]:
import torch
from transformers import AutoModel, AutoTokenizer

word_tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-word")
input_ids = word_tokenizer(prompt, return_tensors='pt')['input_ids']
input_ids.size()

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at C:\Users\F4L5GPL\.cache\huggingface\hub\models--vinai--bartpho-word\snapshots\bee56ed0f2723cf4510ad3775cf4c5a38eb5e6fa\config.json
Model config MBartConfig {
  "_name_or_path": "vinai/bartpho-word",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "MBartModel"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "max_position_embeddings": 1024,
  "model_ty

torch.Size([1, 9])

In [92]:
input_ids = word_tokenizer(prompt, padding= "max_length", max_length= 1024, return_tensors='pt')['input_ids']
input_ids.size()

torch.Size([1, 1024])

In [93]:
word_tokenizer.decode(input_ids[0])

'<s> Bệnh nhân tên là gì? </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pa

In [94]:
attention_mask = 1 - input_ids

In [95]:
# 1 256 768
language_emb = language_model(input_ids, attention_mask)
print(language_emb.size())

torch.Size([1, 1024, 64001])


In [ ]:
emb

In [79]:
import json

class SQMultiModalProjector(nn.Module):
    def __init__(self, config= None):
        super().__init__()
        if config == None:
            config = json.load(open("projector_config.json"))
        self.linear = nn.Linear(config['vision_config_hidden_size'], config['vision_config_projection_dim'], bias=True)

    def forward(self, image_features):
        # [Batch_Size, Num_Patches, Embed_Dim] -> [Batch_Size, Num_Patches, Projection_Dim]
        hidden_states = self.linear(image_features)
        return hidden_states
    
projector_layer = SQMultiModalProjector()

In [80]:
projection_emb = projector_layer(vision_emb)

In [81]:
projection_emb.size()

torch.Size([1, 256, 1024])

In [85]:
projection_emb

tensor([[[-0.2188, -1.4040, -0.7988,  ...,  0.2214,  1.7730,  0.0639],
         [-0.7899,  0.0579, -2.5591,  ..., -2.1431, -0.0903,  0.2951],
         [-0.6725,  2.1156,  0.2242,  ..., -1.1534,  0.0605, -0.6022],
         ...,
         [ 0.8656, -0.9013, -2.9922,  ..., -0.9108,  0.7782,  1.1648],
         [ 1.3474,  2.0274, -3.1988,  ...,  1.3605,  1.0697,  0.1277],
         [ 0.8628, -0.9647, -1.1414,  ..., -0.6360,  1.1736,  0.9398]]],
       grad_fn=<ViewBackward0>)

In [82]:
word_tokenizer.pad_token_id

1

In [83]:
_, _, embed_dim = projection_emb.shape
print(embed_dim)

batch_size, sequence_length = input_ids.shape
dtype, device = language_emb.dtype, language_emb.device

scaled_image_features = projection_emb / (1024**0.5) #elf.config.hidden_size 1024
final_embedding = torch.zeros(batch_size, sequence_length, embed_dim, dtype=language_emb.dtype, device=language_emb.device)
final_embedding

1024


tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])

In [84]:
final_embedding.size()

torch.Size([1, 1024, 1024])

In [88]:
cat_emb = torch.cat([projection_emb, language_emb], dim = 1)[:, :1024, :]
cat_emb.size()

torch.Size([1, 1024, 1024])

In [108]:
import pandas as pd

df = pd.read_csv("csv_data/test.csv")
df.head()

,Unnamed: 0,question,answer,img_id,type
0,0,màu của miếng vá là gì,màu xanh dương,557067,2
1,1,màu của áo là gì,màu cam,436394,2
2,2,màu của áo là gì,màu xanh dương,541050,2
3,3,màu của căn phòng là gì,màu trắng,426077,2
4,4,màu của quả bóng là gì,màu đỏ,314710,2


In [109]:
type(list(df['img_id'])[0])

int

In [110]:
from os import listdir
from os.path import isfile, join

image_train = [int(f.split(".")[0]) for f in listdir("images/test/") if isfile(join("images/test/", f))]
image_train[:3]

[1271, 1436, 1626]

In [111]:
print(len(df))
df = df[df['img_id'].isin(image_train)]
print(len(df))

3001
23


In [112]:
df.to_csv("csv_data/test.csv", index =False)

In [113]:
import pandas as pd

df_train = pd.read_csv("csv_data/train.csv")
df_test = pd.read_csv("csv_data/test.csv")

In [114]:
answer = list(df_train['answer']) + list(df_test['answer'])

In [ ]:
from collections import defaultdict

vocab_answer = defaultdict(lambda: len(vocab_answer))
answer = list(set(answer))

for a in answer:
    vocab_answer.update()